In [202]:
__author__ = 'Arwa Shaker'
"""this Algorithm takes a task and a worker and return the match score between thier skills"""
""" This is the final algorithm the one that we use the level score to reason about the worker skill"""


' This is the final algorithm the one that we use the level score to reason about the worker skill'

In [203]:
import numpy as np
import pandas as pd 
import ast

In [204]:
def is_nan(x):
    return (x is np.nan or x != x)

## the concept similairty method -- Taxonomy based simialrity

In [262]:
from ete3 import TreeNode,Tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def concept_similarity_measure_ex2(C1, C2):
    
    taxonomy = Tree("skills_small_taxonomy_tree_level_score.nw")
    # print("C1",C1,"\n","C2",C2)
    

    N1 = 0 # the distance from Concept 1 to the least common subsumer
    N2 = 0 # the distance from Concept 2 to the least common subsumer
    N = 0  # the distance from  the least common subsumer to the root
    """ -----------------L   he shortest path between the tow concepts------------------------------"""


    node1 = taxonomy.search_nodes(name=C1)
    node2 = taxonomy.search_nodes(name=C2)
    # print('node1', node1)
    # print('node2', node2)
    # if the skill is not found in the taxonomy
    if node1 == [] or node2 == []:
        # print('skills not in taxonomy')
        # print(C1, C2)
        data = [C1, C2]

        # Vectorise the data
        vec = TfidfVectorizer()

        X = vec.fit_transform(
            data)  # `X` will now be a TF-IDF representation of the data, the first row of `X` corresponds to the first sentence in `data`

        # Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
        S = cosine_similarity(X)
        similarity = S[0, 1]


        # print('simmmms',similarity)
        l1 = l2 = 1.0  # how much should it be

    else:
        node1 = node1[0]
        node2 = node2[0]
        common = node1.get_common_ancestor(node2)
        # print(common.is_root())
        # print("common is ",common.name)

        """ ------------------N the distance from root node to the least common subsumer-------------------"""
        root = taxonomy.get_tree_root()
        N = taxonomy.get_distance(common, root, topology_only=False)
        # print("N = the distance between the common ancestor", common.name, "AND  ROOT IS ", N)

        """ ----------------N1 the distance from Concept 1 to the least common subsumer--------------------"""
        N1 = taxonomy.get_distance(C1, common, topology_only=False)
        # print("N1 = the distance between",C1,"AND  ROOT IS ",N1)

        """ ----------------N1 the distance from Concept 2 to the least common subsumer--------------------"""
        N2 = taxonomy.get_distance(C2, common, topology_only=False)
        # print("N2 = the distance between",C2, "AND  ROOT IS ", N2)

        """ -------------------------------COMPUTE THE MEASURE FORMULA----------------------------------------"""

        similarity = (2*N)/(N1+N2+(2*N))

        
        l1 = node1.level_score
        l2 = node2.level_score
        similarity = round(similarity, 2)
#         print("similarity between ", C1,"and",C2, "is", similarity)
#         print("level score  ",C1,'  is', l1)
#         print("level score  ",C2,'  is', l2)
#         print("---------------------------------------------------")
    return similarity,l1,l2 

In [297]:
from IPython.display import display, HTML
def Algorithm2_Get_task_to_worker_skill_matching2():


    skills_t =['Programming', 'Graphic Design','HTML','SQL','Javascript']

    skills_w = [('HTML5', 0.8), ('MySQL', 0.9), ('Banner Design', 1.0), ('Plgin', 1.0), ('Linux', 0.4), ('C++ Programming', 0.5)]

    print('task skills \n', skills_t)
    print('worker skills \n', skills_w)


    max_sim_list = []
    weight_list = []
    
    """compare every skill in the task by all of the skills in the worker """ 
    
    for st in skills_t:
        sim_list = pd.DataFrame()
        if not is_nan(st):
            for sw in skills_w:
                if sw[0] != '':
                    sim,lt,lw = concept_similarity_measure_ex2(st,sw[0])
                
                sim_list.loc[st,sw[0]] = sim 
                
                display("sim list before", sim_list)
                
                if sim >= 0.8:
                    if lt <= lw:
                        sim = 1 
                sim_list.loc[st,sw[0]] = sim 

        if not sim_list.empty:
            display("sim list", sim_list)
            max_sim = sim_list.loc[st].max()  # take the maximum similarity 
            skill = sim_list.loc[st].idxmax(axis=0) #find the skill with the maxomum similarity to get the conf value of it
            print('maximum skill', skill)
            
            """ find the skill with the maximum similarity """
            for sw in skills_w:
                print('sw[0]', sw[0], 'skill',skill)
                if sw[0] is skill: 
                    worker_conf = float(sw[1])
                    print("yes worker conf in", sw[0],"is",worker_conf)
                    break
           
            
            
            max_sim= max_sim * worker_conf  # multiply by the confident score
            max_sim_list.append(max_sim)
            weight_list.append(worker_conf)

    print('max list', max_sim_list)
    print('max_sim',sum(max_sim_list))
    print('sum(weight_list)', sum(weight_list))
    sum_max = sum(max_sim_list)
    sum_weight = sum(weight_list)

    if sum_weight != 0:
        skill_match_score = sum_max / sum_weight
    else:
        skill_match_score = 0.0
    print('skill_match_score',skill_match_score)
    return skill_match_score

In [298]:
Algorithm2_Get_task_to_worker_skill_matching2()


task skills 
 ['Programming', 'Graphic Design', 'HTML', 'SQL', 'Javascript']
worker skills 
 [('HTML5', 0.8), ('MySQL', 0.9), ('Banner Design', 1.0), ('Plgin', 1.0), ('Linux', 0.4), ('C++ Programming', 0.5)]


'sim list before'

,HTML5
Programming,0.29


'sim list before'

,HTML5,MySQL
Programming,0.29,0.33


...

,HTML5,MySQL,Banner Design,Plgin,Linux,C++ Programming
Javascript,0.67,0.25,0.29,0.6,0.29,0.25


'sim list'

,HTML5,MySQL,Banner Design,Plgin,Linux,C++ Programming
Javascript,0.67,0.25,0.29,0.6,0.29,0.25


maximum skill HTML5
sw[0] HTML5 skill HTML5
yes worker conf in HTML5 is 0.8
max list [0.4, 1.0, 0.8, 0.9, 0.536]
max_sim 3.636
sum(weight_list) 3.9000000000000004
skill_match_score 0.9323076923076923


0.9323076923076923